# FRED Series

This documentation serves as an introduction to requesting series data from the FRED API with `pyfredapi`.

The primary use case of the FRED web service is to pull economic series data for analysis or reporting. The `pyfredapi.series` module aims to make it easy to get series metadata and data from [FRED](https://fred.stlouisfed.org/) or [ALFRED](https://alfred.stlouisfed.org/).

The following examples illustrate usage of the Series endpoint functions in `pyfredapi`.

## Setup

Import `pyfredapi`

In [1]:
import pyfredapi as pf
from rich import print as rprint
from rich.pretty import pprint

## Series Metadata

Before looking at series data, it is helpful to understand the `pyfredapi`'s `get_series_info` function and `SeriesInfo` object. Each series published by FRED has associated metadata such as:

- Start and end dates
- Publish frequency
- Unit of measure

You can query a series' information directly with `get_series_info`. The `get_series_info` function returns a `SeriesInfo` object that contains all the metadata for the given series.

In the below example, we request information for the U.S. GDP series. From the result, we can see that the GDP series is

* Published quarterly
* The earliest data available is 1947-01-01
* The unit of measure is Billions of Dollars

In [2]:
gdp_info = pf.get_series_info(series_id="GDP")

# Using rich to pretty print the SeriesInfo
rprint(gdp_info)

SeriesInfo(
    id='GDP',
    realtime_start='2023-04-27',
    realtime_end='2023-04-27',
    title='Gross Domestic Product',
    observation_start='1947-01-01',
    observation_end='2023-01-01',
    frequency='Quarterly',
    frequency_short='Q',
    units='Billions of Dollars',
    units_short='Bil. of $',
    seasonal_adjustment='Seasonally Adjusted Annual Rate',
    seasonal_adjustment_short='SAAR',
    last_updated='2023-04-27 07:53:02-05',
    popularity=93,
    notes='BEA Account Code: A191RC\n\nGross domestic product (GDP), the featured measure of U.S. output, is the 
market value of the goods and services produced by labor and property located in the United States.For more 
information, see the Guide to the National Income and Product Accounts of the United States (NIPA) and the Bureau 
of Economic Analysis (http://www.bea.gov/national/pdf/nipaguid.pdf).'
)

`SeriesInfo` also provides a useful method to open the FRED webpage for the series. Call `open_url` on the info object and a new browser tab will open.

In [3]:
gdp_info.open_url()

## Pull data

The `get_series` function gets the latest data available for a given series. The default return for is a pandas dataframe. The `get_series` function also accepts a `return_format` argument that can be set to `json` to return the data in a json-like list of dictionaries.

### Pandas Dataframe

In [4]:
gdp_df = pf.get_series(series_id="GDP")

In [5]:
gdp_df.tail()

,realtime_start,realtime_end,date,value
304,2023-04-27,2023-04-27,2022-01-01,24740.480
305,2023-04-27,2023-04-27,2022-04-01,25248.476
306,2023-04-27,2023-04-27,2022-07-01,25723.941
307,2023-04-27,2023-04-27,2022-10-01,26137.992
308,2023-04-27,2023-04-27,2023-01-01,26465.865


When the data is returned as a pandas dataframe the correct data type will be inferred and applied to each column.

In [6]:
gdp_df.dtypes

realtime_start            object
realtime_end              object
date              datetime64[ns]
value                    float64
dtype: object

### Json

In [7]:
gdp_json = pf.get_series(series_id="GDP", return_format="json")

In [8]:
pprint(gdp_json, max_length=4)

[
│   {'realtime_start': '2023-04-27', 'realtime_end': '2023-04-27', 'date': '1946-01-01', 'value': '.'},
│   {'realtime_start': '2023-04-27', 'realtime_end': '2023-04-27', 'date': '1946-04-01', 'value': '.'},
│   {'realtime_start': '2023-04-27', 'realtime_end': '2023-04-27', 'date': '1946-07-01', 'value': '.'},
│   {'realtime_start': '2023-04-27', 'realtime_end': '2023-04-27', 'date': '1946-10-01', 'value': '.'},
│   ... +305
]

## Data revisions

Many economic series are often revised as more complete information is made available. Every time a new version of a data series is released, FRED displays the latest version, and the replaced version is archived in [ALFRED](https://alfred.stlouisfed.org/). ALFRED stores all the previous versions of data so that it possible to understand, "what was known when?".

`pyfredapi` provides functions to access data from ALFRED:

* `get_series_all_releases` - get data for all releases
* `get_series_initial_release` - get data for the initial release
* `get_series_asof_date` - get data released on or before a specific date

### Get all releases

`get_series_all_releases` returns all the observations for each release of an economic series. In the example below, a request is made for all the releases of Gross Domestic Product. We can see that the GDP estimates for Q2 2022 has been revised 3 times. The first release was on 2022-07-28. Since then, two revisions have been released on 2022-08-25 and 2022-09-29.

The realtime columns indicates the time period that the data was known to be true. For more information, see the [FRED docs](https://fred.stlouisfed.org/docs/api/fred/realtime_period.html) on realtime periods.

In [9]:
gdp_all_releases_df = pf.get_series_all_releases("GDP")
gdp_all_releases_df.tail()

,realtime_start,realtime_end,date,value
3100,2022-12-22,9999-12-31,2022-07-01,25723.941
3101,2023-01-26,2023-02-22,2022-10-01,26132.458
3102,2023-02-23,2023-03-29,2022-10-01,26144.956
3103,2023-03-30,9999-12-31,2022-10-01,26137.992
3104,2023-04-27,9999-12-31,2023-01-01,26465.865


### Get initial release

`get_series_initial_release` return only the first release of the series. Below we see that only first estimate of Q2 2022 GDP released on 2022-07-28 is included in the dataframe.

In [10]:
gdp_initial_release_df = pf.get_series_initial_release("GDP")
gdp_initial_release_df.tail()

,realtime_start,realtime_end,date,value
121,2022-04-28,2022-05-25,2022-01-01,24382.683
122,2022-07-28,2022-08-24,2022-04-01,24851.809
123,2022-10-27,2022-11-29,2022-07-01,25663.289
124,2023-01-26,2023-02-22,2022-10-01,26132.458
125,2023-04-27,9999-12-31,2023-01-01,26465.865


### Get releases as-of date

`get_series_asof_date` returns all releases of a series made on or before a given date. This is helpful if you want limit your analysis window to only the data know on or before a given date.

For example, suppose we want the GDP estimates available on or before 2022-09-01. We can use `get_series_asof_date` with the date `2022-09-01`. The response includes the Q2 2022 estimates for 2022-07-28 and 2022-08-25, but not 2022-09-29 since that is after 2022-09-01.

In [11]:
gdp_090122_df = pf.get_series_asof_date("GDP", date="2022-09-01")
gdp_090122_df.tail()

,realtime_start,realtime_end,date,value
3071,2022-04-28,2022-05-25,2022-01-01,24382.683
3072,2022-05-26,2022-06-28,2022-01-01,24384.289
3073,2022-06-29,2022-09-01,2022-01-01,24386.734
3074,2022-07-28,2022-08-24,2022-04-01,24851.809
3075,2022-08-25,2022-09-01,2022-04-01,24882.878


## Additional Parameters

You can pass additional arguments to the FRED API via `**kwargs`. The FRED API endpoint supports many parameters to customize your query. See the [FRED API docs](https://fred.stlouisfed.org/docs/api/fred/) for details.

In [12]:
extra_parameters = {
    "observation_start": "2020-01-01",
    "observation_end": "2020-12-31",
}

gdp_df = pf.get_series(series_id="GDP", **extra_parameters)

In [13]:
gdp_df

,realtime_start,realtime_end,date,value
0,2023-04-27,2023-04-27,2020-01-01,21538.032
1,2023-04-27,2023-04-27,2020-04-01,19636.731
2,2023-04-27,2023-04-27,2020-07-01,21362.428
3,2023-04-27,2023-04-27,2020-10-01,21704.706
